In [1]:
import pandas as pd
from services.db_service import DbService

In [2]:
personal_pool_collection = DbService().personal_pool_collection()
global_pool = DbService().pool_collection()

In [5]:
df = pd.read_csv("../raw-data/ligamagic.csv", encoding="ISO-8859-1")
df.head()

,Edicao (PTBR),Edicao (EN),Edicao (Sigla),Card (PT),Card (EN),Quantidade,Qualidade (M NM SP MP HP D),Idioma (BR EN DE ES FR IT JP KO RU TW),Raridade (M R U C),Cor (W U B R G M A L),Extras,Card #,Comentario
0,NaN,Dominaria Remastered,dmr,Fogo // Gelo,Fire // Ice,2,M,EN,U,M,Foil,215,NaN
1,NaN,Dominaria Remastered (Retro Frame),rfdmr,Recuo,Recoil,3,M,EN,U,M,Foil,365,NaN
2,NaN,Double Masters,2xm,Amuleto Izzet,Izzet Charm,4,M,EN,U,M,Foil,202,NaN
3,NaN,Double Masters 2022,2x2,Coatl Alado,Winged Coatl,2,M,EN,C,M,Foil,295,NaN
4,NaN,Double Masters 2022,2x2,Coatl Alado,Winged Coatl,2,NM,EN,C,M,NaN,295,NaN


In [6]:
df = df[["Card (EN)", "Quantidade"]]
df = df.rename(columns={"Quantidade": "count", "Card (EN)": "name"})

df.head()

,name,count
0,Fire // Ice,2
1,Recoil,3
2,Izzet Charm,4
3,Winged Coatl,2
4,Winged Coatl,2


In [7]:
selected_columns = df[["count", "name"]]
cards = selected_columns.to_dict(orient="records")
cards

[{'count': 2, 'name': 'Fire // Ice'},
 {'count': 3, 'name': 'Recoil'},
 {'count': 4, 'name': 'Izzet Charm'},
 {'count': 2, 'name': 'Winged Coatl'},
 {'count': 2, 'name': 'Winged Coatl'},
 {'count': 4, 'name': 'Agony Warp'},
 {'count': 4, 'name': 'Bloodwater Entity'},
 {'count': 4, 'name': 'Burning-Tree Emissary'},
 {'count': 4, 'name': 'Terminate'},
 {'count': 2, 'name': 'Fire // Ice'},
 {'count': 1, 'name': 'Unmake'},
 {'count': 1, 'name': 'Unmake'},
 {'count': 4, 'name': 'Branching Bolt'},
 {'count': 4, 'name': 'Armadillo Cloak'},
 {'count': 1, 'name': 'Nivix Cyclops'},
 {'count': 3, 'name': 'Blightning'},
 {'count': 3, 'name': 'Unmake'},
 {'count': 4, 'name': "Nature's Chant"},
 {'count': 4, 'name': 'Breathless Knight'},
 {'count': 1, 'name': 'Fire // Ice'},
 {'count': 4, 'name': 'Foundry Helix'},
 {'count': 4, 'name': 'Goblin Electromancer'},
 {'count': 2, 'name': 'Dinrova Horror'},
 {'count': 4, 'name': 'Wild Cantor'},
 {'count': 3, 'name': 'Safehold Elite'},
 {'count': 4, 'name':

In [8]:
import_type = "replace"  # sum or replace

def add_or_update_card(card):
    card_from_pool = global_pool.find_one({"scryfall_name": card["name"]})
    name = card_from_pool["name"] if card_from_pool else card["name"]

    existing_card = personal_pool_collection.find_one({"scryfall_name": card["name"]})
    if not existing_card:
        personal_pool_collection.insert_one(
            {"name": name, "scryfall_name": card["name"], "count": card["count"]})
        return

    if import_type == "sum":
        card["count"] += existing_card["count"]

    personal_pool_collection.update_one({"_id": existing_card["_id"]}, {
                          "$set": {"count": card["count"]}})

In [9]:
for card in cards:
    add_or_update_card(card)